In [ ]:
import pandas as pd
import numpy as np

In [ ]:
institute = pd.read_csv('../Dataset/edu/서울시 학원 데이터 전처리.csvv')
institute = institute[['학원명', '도로명주소']]
institute = institute.drop_duplicates()
institute = institute.reset_index(drop=True)
institute

In [ ]:
addrs = institute['도로명주소']

In [ ]:
import requests
import json

locations = []
for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

In [ ]:
city = [] ## 구
town = [] ## 행정동
for i in range(len(locations)):
    
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))
        
    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')
    
    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

In [ ]:
city_town = np.array([city,town]).T
df_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
df_temp.head()

In [ ]:
institute['구'] = df_temp['region_2depth_name']
institute['동'] = df_temp['region_3depth_h_name']
institute.head()

In [ ]:
# 데이터 타입 변환

institute['구'] = institute['구'].astype(str)
institute['동'] = institute['동'].astype(str)

In [ ]:
in_na = institute[institute['동'] == '없음']
in_na = in_na.reset_index(drop=True)
in_na

In [ ]:
in_na = in_na.dropna()
in_na

In [ ]:
addrs2 = in_na['도로명주소']
addrs2

In [ ]:
in_na.to_csv('../Dataset/edu/오류값.csv', index=False)

In [ ]:
from geopy.geocoders import Nominatim

list = []

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

In [ ]:
for i in range(133):
    try:
        crd = geocoding(addrs2[i])
        if crd is not None:
            list.append(crd)
        else:
            list.append({"lat": 0, "lng": 0})            # 결측치 0기재
            print('{} 번째 좌표를 불러오지 못함'.format(i))
    
        if i % 10 == 0:
            print('{} 번째 좌표 정보 불러오는 중...'.format(i))

    except Exception as e:
        list.append({"lat": 0, "lng": 0})                # 결측치 0기재
        print('{} 번째 데이터 오류 발생'.format(i))

In [ ]:
add = pd.DataFrame(list)
in_na['x'] = add['lng']
in_na['y'] = add['lat']
in_na

In [ ]:
error = pd.read_csv('../Dataset/edu/오류값.csv')
error.columns

In [ ]:
error = error.rename(columns= {'학원명': '학원명',
                               '도로명주소': '도로명주소',
                               '구': '구',
                               '동': '동',
                               'x': 'x',
                               'y': 'y'})

In [ ]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in error.iterrows():
    get_location(row)
    
    # 20개씩 처리될 때 메시지 출력
    if i % 20 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
error['구'] = [location.get('gu') for location in locations]
error['동'] = [location.get('dong') for location in locations]


In [ ]:
error.columns

In [ ]:
institute.info

In [ ]:
institute

In [ ]:
error.info()
error2 = error[['학원명', '도로명주소', '구', '동']]

In [ ]:
inst2 = institute[institute['동'] != '없음']
inst2 = institute[institute['구'] != '없음']
inst2.info()

In [ ]:
inst_new = pd.concat([inst2, error2], axis=0)
inst_new.info()

In [ ]:
inst_new = inst_new.reset_index(drop=True)
inst_new


In [ ]:
inst = inst_new[inst_new['동'] != 'None']

In [ ]:
in3 = inst.groupby(['구', '동'], as_index=False)['학원명'].count()
in3

In [ ]:
in3.to_csv('../Dataset/edu/교육_학교 수_동별_2023.csv', index=False)